In [18]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from numba import njit,jit,typed


In [29]:
@njit
def oscillatorModelOde(Y, t, can): 

    #V = can[12]
    VA = can[12]
    #A = V/VA
    sigma = 0.001
    #y1 = VA/(2*sigma)
    
    
    ka1 = can[0]
    kb1 = can[1]
    kcat1 = can[2]

    ka2 = can[3]
    kb2 = can[4]

    ka3 = can[5]
    kb3 = can[6]

    ka4 = can[7]
    kb4 = can[8]

    ka7 = can[9]
    kb7 = can[10]
    kcat7 = can[11]

    ka5 = can[9]
    kb5 = can[10]
    kcat5 = can[11]

    ka6 = can[0]
    kb6 = can[1]
    kcat6 = can[2]


    # ka6 = can[12] 
    # kb6 = can[13]
    # kcat6 = can[14]

    
    #initial conditions
    L = Y[0]   # make sure in uM
    Lp = Y[1]
    K = Y[2]
    P = Y[3]
    LK = Y[4] 
    A = Y[5]
    LpA = Y[6] 
    LpAK = Y[7] 
    LpAP = Y[8] 
    LpAPLp = Y[9] 
    LpAKL = Y[10] 
    LpP = Y[11] 

    dL = (kb1*LK) - (ka1*L*K) + (kcat5*LpAPLp) + (kb6*LpAKL) - ((VA/(2*sigma))*ka6*LpAK*L) + (kcat7*LpP)
    dLp = (kcat1*LK) + (kb2*LpA) - (ka2*Lp*A) + (kb5*LpAPLp) - ((VA/(2*sigma))*ka5*Lp*LpAP) + (kcat6*LpAKL) - (ka7*Lp*P) + (kb7*LpP)
    dK = (kb1*LK) - (ka1*L*K) + (kcat1*LK) + (kb3*LpAK) - (ka3*LpA*K)
    dP = (kb4*LpAP) - (ka4*LpA*P) - (ka7*Lp*P) + (kb7*LpP) + (kcat7*LpP)
    dLK = (ka1*L*K) - (kb1*LK) - (kcat1*LK)
    dA = (kb2*LpA) - (ka2*Lp*A)
    dLpA = (ka2*Lp*A) - (kb2*LpA) + (kb3*LpAK) - (ka3*LpA*K) + (kb4*LpAP) - (ka4*LpA*P)
    dLpAK = (ka3*LpA*K) - (kb3*LpAK) + (kb6*LpAKL) - ((VA/(2*sigma))*ka6*LpAK*L) + (kcat6*LpAKL)
    dLpAP = (ka4*LpA*P) - (kb4*LpAP) + (kb5*LpAPLp) - ((VA/(2*sigma))*ka5*LpAP*Lp) + (kcat5*LpAPLp)
    dLpAPLp = ((VA/(2*sigma))*ka5*LpAP*Lp) - (kb5*LpAPLp) - (kcat5*LpAPLp)
    dLpAKL = ((VA/(2*sigma))*ka6*LpAK*L) - (kb6*LpAKL) - (kcat6*LpAKL)
    dLpP = (ka7*Lp*P) - (kb7*LpP) - (kcat7*LpP)

    #Y[0],Y[1],Y[2],Y[3],Y[4],Y[5],Y[6],Y[7],Y[8],Y[9],Y[10],Y[11] = dL,dLp,dK,dP,dLK,dA,dLpA,dLpAK,dLpAP,dLpAPLp,dLpAKL,dLpP

    return([dL, dLp, dK, dP, dLK, dA, dLpA, dLpAK, dLpAP, dLpAPLp, dLpAKL, dLpP])	
    #return Y

In [30]:
t = np.linspace(0,100,num=10000)

candidate = typed.List([0.5399439956871153,
 32.49731125365913,
 459.0713154241908,
 0.6703697205723796,
 98.55386776137301,
 66.94761309866233,
 187.88476784879947,
 0.732829082837601,
 0.5932803922080772,
 0.8833353263653272,
 79.8349667706061,
 98.51067194455857,
 1.21704450980531,
 8.973816043747753,
 0.6170991018130821,
 0.7934153177539267,
 2.6856696379362606])

initials = [candidate[13],0,candidate[14],
            candidate[15],0,candidate[16],0,0,
            0,0,0,0]

%timeit sol = odeint(oscillatorModelOde,initials,t,args=(candidate,))



47 ms ± 1.29 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
from scipy.special import gamma, airy
y1_0 = 1.0 / 3**(2.0/3.0) / gamma(2.0/3.0)
y0_0 = -1.0 / 3**(1.0/3.0) / gamma(1.0/3.0)
y0 = [y0_0, y1_0]
t = np.arange(0, 4.0, 0.01)


In [13]:
from numba import jit
@jit
def RHS(y, t):
    y[0],y[1] = t*y[1],y[0]
    return y

%timeit odeint(RHS, y0, t)

101 µs ± 1.69 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [21]:
initials = tf.constant([candidate[13],0.0,candidate[14],candidate[15],0.0,candidate[16],0.0,0.0,0.0,0.0,0.0,0.0])
candidate = tf.constant(candidate)


In [32]:
t_begin = tf.constant(0.0)
t_end = tf.constant(100.0)

In [33]:
sol = tfp.math.ode.BDF().solve(oscillatorModelOde, t_begin, initials, solution_times=tfp.math.ode.ChosenBySolver(t_end),constants={'can':candidate})